# TP 3 : Programmation Dynamique

### Initialisation (à faire une seule fois)

In [1]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
  Installed Netpbm ───────────── v1.1.0
  Installed GraphRecipes ─────── v0.5.5
  Installed ImageAxes ────────── v0.6.10
  Installed GraphPlot ────────── v0.5.2
  Installed StatsBase ────────── v0.33.21
  Installed IJulia ───────────── v1.23.3
  Installed Compose ──────────── v0.9.4
  Installed ImageMetadata ────── v0.9.8
  Installed ImageBase ────────── v0.1.5
  Installed GeometryBasics ───── v0.3.10
  Installed ImageCore ────────── v0.9.4
  Installed NetworkLayout ────── v0.2.0
  Installed ColorSchemes ─────── v3.20.0
  Installed TranscodingStreams ─ v0.9.10
  Installed MozillaCACerts_jll ─ v2022.10.11+0
  Installed StructArrays ─────── v0.4.2
  Installed Parsers ──────────── v2.4.2
  Installed Interpolations ───── v0.12.10
######################################################################### 100,0%
   Updating `~/.julia/environments/v1.4/Project.toml`
  [13f3f980] ↓ CairoMakie v0.7.2 ⇒ v0.6.4
 

### Récupération des données

In [9]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Création du tableau avec la relation de récurrence et Recherche de la solution

In [13]:
#=
function creationTab(cout, poids, capacite)
    nbObjets = length(cout) # length(poids)
    tab = zeros(Int, nbObjets, capacite+1)

    for j in 2:(capacite+1)
        if poids[1] <= j-1
            tab[1,j]=cout[1]
        end
    end

    for i=2:nbObjets 
        for j=2:(capacite+1)
            if (j-poids[i])>0
                tab[i,j]= max(tab[i-1,j], tab[i-1, j-poids[i]] + cout[i] )
            else
                tab[i,j]= tab[i-1,j]
            end
        end
    end
    return tab
end
=#

function creationTabandObjList(cout, poids, capacite)
    nbObjets = length(cout) # length(poids)
    tab = zeros(Int, nbObjets, capacite+1)
    
    # objetList[i,j] liste d'objet utilisé pour atteindre la capacité tab[i,j]
    objList = [[] for i=1:nbObjets, j=1:(capacite+1)]
    
    for j in 2:(capacite+1)
        if poids[1] <= j-1
            tab[1,j]=cout[1]
        end
    end

    for i=2:nbObjets 
        for j=2:(capacite+1)
            if (j-poids[i])>0
                # tab[i,j]= max(tab[i-1,j], tab[i-1, j-poids[i]] + cout[i])
                cij_1 = tab[i-1,j]
                cij_2 = tab[i-1, j-poids[i]] + cout[i]
                
                if cij_1 > cij_2
                    tab[i,j] = cij_1
                    
                    objList_1 = copy(objList[i-1,j])
                    objList[i,j] = objList_1
                else
                    tab[i,j] = cij_2
                    
                    objList_2 = copy(objList[i-1,j-poids[i]])
                    objList_2 = push!(objList_2,i)
                    objList[i,j] = objList_2
                end 
            else
                tab[i,j]= tab[i-1,j]
                objList[i,j] = objList[i-1,j]
            end
        end
    end
    return tab, objList
end

function trouverSol(tab, objList, nbObjets, capacite)
    coutMax = 0
    objListMax = []
    
    for i in 1:nbObjets
        tmp = tab[i,capacite+1]
        objList_tmp = objList[i,capacite+1]
        if tmp > coutMax
            coutMax = tmp
            objListMax =objList_tmp
        end
    end
    
    return coutMax, objListMax
end

trouverSol (generic function with 2 methods)

## Boucle Principale : Résolution du problème du Sac à Dos

In [18]:
function SolveProblemeSacADos(filename)
    # Récupération des données
    price, weight, capacity = readKnaptxtInstance(filename)
    
    # Ordre décroissant suivant les prix
    indSorted = sortperm(price)
    indSorted = reverse(indSorted)
    
    priceSorted = price[indSorted]
    weightSorted = weight[indSorted]
    
    # Affichage des Donnees
    println("Donnees :")
    println("Prix:", priceSorted)
    println("\nPoids:", weightSorted)
    
    array, objList = creationTabandObjList(priceSorted, weightSorted, capacity)
    
    # Affichage du tableau crée si sa taille n'est pas très grande 
    if length(array) < 50
        println("\nTableau crée avec la fonction creationTab: array")
        println(array)
    end
        
    nbObjets = length(price)
    BestProfit, objListMax = trouverSol(array, objList, nbObjets, capacity)
    
    BestSol = zeros(Int, nbObjets, 1)
    
    indObjetsChoisis = indSorted[objListMax]
    
    for i in indObjetsChoisis
        BestSol[i] = 1 
    end
    
    return BestProfit, BestSol
end

SolveProblemeSacADos (generic function with 1 method)

### Commentaires du Code : Fonctionnement de l'algorithme

Nous avons utilisé l'algorithme de programmation dynamique présenté dans le cours pour résoudre le problème du sac à dos. Cet algorithme consiste tout d'abord en la création de deux tableaux. Nous avons complété le premier en utilisant la relation de récurrence fournie en cours et le second avec les identifiants des objets selectionnés pour atteindre le coût inscrit dans la case correspondante du premier tableau (creationTabandObjList). Pour faire cela, nous avons tout d'abord commencé par classer les listes des objets et des prix suivant l'ordre décroissant des prix. 

Une fois le tableau créé, nous avons sélectionné la dernière colonne du tableau et nous cherchons le coût maximal. La case correspondante à celle-là dans le second tableau nous donne la liste des objets selectionnés pour atteindre ce coût. Nous obtenons ainsi la solution du problème.

###  Analyses Demandées

### Test Instance 1 :
Courte argumentation de l'adéquation du résultat avec cette instance.
- Solution Obtenue
- Valeurs de la Fonction-Objectif

#### Affichage du résultat final avec cette instance :

In [19]:
instance = "instances/instance_exemple_cours.txt"

BestProfit, BestSol = SolveProblemeSacADos(instance)
println("\n---------------------------------------------------\n")
println("BestProfit = ", BestProfit, "\nBestSol x=", BestSol)

Donnees :
Prix:Any[42, 40, 25, 12]

Poids:Any[7, 4, 5, 3]

Tableau crée avec la fonction creationTab: array
[0 0 0 0 0 0 0 42 42 42 42; 0 0 0 0 40 40 40 42 42 42 42; 0 0 0 0 40 40 40 42 42 65 65; 0 0 0 12 40 40 40 52 52 65 65]

---------------------------------------------------

BestProfit = 65
BestSol x=[0; 1; 0; 1]


### Test Instance 2 :
Courte argumentation de l'adéquation du résultat avec cette instance.
- Solution Obtenue
- Valeurs de la Fonction-Objectif

#### Affichage du résultat final avec cette instance :

In [20]:
# Pensez à commenter les affichages des données si très grande instances
instance = "instances/knapPI_16_20_1000_1_-2291.opb.txt"

BestProfit, BestSol = SolveProblemeSacADos(instance)
println("\n---------------------------------------------------\n")
println("BestProfit = ", BestProfit, "\nBestSol x=", BestSol)

Donnees :
Prix:Any[1152, 1151, 1110, 1074, 1062, 1049, 994, 983, 950, 914, 886, 848, 839, 819, 814, 762, 737, 629, 403, 111]

Poids:Any[994, 992, 893, 815, 791, 766, 667, 649, 598, 544, 506, 457, 446, 422, 416, 359, 334, 237, 94, 7]

---------------------------------------------------

BestProfit = 2291
BestSol x=[0; 0; 1; 0; 0; 1; 0; 0; 0; 0; 0; 0; 0; 0; 1; 0; 1; 0; 0; 0]


### Test Instance 3 :
Courte argumentation de l'adéquation du résultat avec cette instance.
- Solution Obtenue
- Valeurs de la Fonction-Objectif

#### Affichage du résultat final avec cette instance :

In [21]:
# Pensez à commenter les affichages des données si très grande instances
instance = "instances/knapPI_4_100_1000_2_-1041.opb.txt"

BestProfit, BestSol = SolveProblemeSacADos(instance)
println("\n---------------------------------------------------\n")
println("BestProfit = ", BestProfit, "\nBestSol x=", BestSol)

Donnees :
Prix:Any[997, 997, 995, 994, 992, 984, 982, 972, 955, 955, 946, 943, 941, 931, 928, 923, 908, 902, 901, 895, 893, 887, 882, 874, 874, 872, 863, 858, 856, 845, 844, 819, 815, 810, 801, 801, 800, 800, 796, 795, 795, 794, 793, 791, 789, 785, 772, 769, 768, 766, 764, 764, 748, 744, 742, 741, 738, 737, 731, 730, 726, 724, 720, 719, 715, 707, 700, 696, 695, 686, 682, 674, 668, 667, 659, 656, 654, 649, 645, 636, 634, 633, 617, 617, 616, 608, 606, 600, 598, 595, 593, 588, 583, 574, 569, 561, 560, 553, 552, 544, 539, 537, 536, 529, 526, 518, 513, 513, 506, 493, 487, 487, 485, 481, 470, 465, 463, 458, 457, 451, 448, 448, 446, 442, 433, 428, 422, 421, 420, 419, 416, 408, 406, 398, 396, 375, 367, 363, 359, 358, 352, 347, 339, 334, 331, 326, 322, 320, 303, 299, 294, 277, 267, 261, 252, 248, 242, 240, 237, 220, 213, 204, 199, 196, 187, 183, 182, 171, 171, 160, 145, 138, 132, 131, 127, 122, 122, 116, 109, 107, 98, 97, 94, 94, 81, 72, 70, 70, 70, 70, 59, 58, 47, 43, 29, 23, 9, 8, 7, 1]

Poid

### Comparaison avec les Résultats obtenues avec le Branch-and-Bound implémenté lors du TP2

Nous avons effectué des tests avec l'algorithme Branch-and-Bound implémenté lors du TP2 sur les trois instances utilisées ci-dessus. Nous remarquons que nous obtenons les mêmes résultats. L'algorithme utilisé dans le TP3, cependant, semble être plus efficace.